In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from tqdm import tqdm_notebook as tqdm

from esn_module import esn

from echo_state_tests import gen_in_out_one_in_subs, test_XOR

from scipy.sparse import csr_matrix

/home/fschubert/work/py36/lib/python3.6/site-packages/tqdm/autonotebook/__init__.py:18: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [33]:
N = 1000
cf = 0.1
cf_in = 1.

W = np.random.normal(0.,1./(cf*N)**.5,(N,N))*(np.random.rand(N,N) <= cf)
W[range(N),range(N)] = 0.

a = np.ones((N))*0.9

Wa = (W.T*a).T

W = csr_matrix(W)
Wa = csr_matrix(Wa)

w_out = np.random.normal(0.,1.,(N))
w_in = np.random.normal(0.,1.,(N))*(np.random.rand(N) <= cf_in)

T = 100

err = np.zeros((T))

dyda = np.zeros((N,N))

derrda = np.zeros((T,N))

x = np.zeros((T,N))

u_in,u_out = gen_in_out_one_in_subs(T,5)

In [34]:
for t in tqdm(range(1,T)):
    x[t,:] = W.dot(a*x[t-1,:]) + w_in*u_in[t]
    
    dyda = Wa.dot(dyda)
    dyda[range(N),range(N)] += x[t,:]
    
    err[t] = w_out.dot(a*x[t])
    
    derrda[t,:] = w_out.dot(dyda)

In [35]:
-(err*derrda.T).mean(axis=1) @ (w_out*(err*x.T).mean(axis=1))

-2104508.0773475915

In [28]:
(w_out*(err*x.T).mean(axis=1)).shape

(200,)